In [1]:
import pickle
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import re

from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, LSTM, Bidirectional, Dropout, BatchNormalization
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path_read = '/Users/andradea/Documents/GitHub/CRAIS/Embeddings/'

with open(path_read + 'word_index.pkl', 'rb') as f:
    word_index = pickle.load(f)

with open(path_read + 'embeddings_index.pkl', 'rb') as f:
    embeddings_index = pickle.load(f)

In [3]:
path = '/Users/andradea/Documents/cr-engine/datasets/ASAP/'
files = os.listdir(path)
files = [f for f in files if 'CR' in f and '.csv' in f]

In [4]:
file = files[0]
df = pd.read_csv(path + file)
df.head(2)

,item,id,response,score_d1,r1_d1,r2_d1
0,ASAP_CR10,ASAP_CR10_1,white :: A white colored doghouse will make t...,2,2,2
1,ASAP_CR10,ASAP_CR10_2,"dark gray :: i would go with dark gray, so tha...",1,1,1


In [5]:
responses = df.response.tolist()
labels = df.score_d1.values

In [6]:
def clean_text(c):
    c = c.lower()
#     c = re.subn("^rt ", "", c)
    c = re.sub("\W", " ", c)
#     c = regexp_replace(c, "[0-9]", "")
    c = re.sub("\s\s+", " ", c)
    c = c.split(' ') # tokenization...

    return c

length = []
for response in responses:
    response = clean_text(response)
    length.append(len(response))
int(np.ceil(np.mean(length)))

43

In [7]:
max_len = int(np.ceil(np.mean(length)))
vec_size = 150
num_classes = len(df.score_d1.value_counts())

In [8]:
vocabulary = list(word_index.keys())

In [9]:
response_embedded = np.zeros((len(responses), max_len, vec_size))

for i,response in enumerate(responses): 
    response = clean_text(response)
    for j,word in enumerate(response):
        if j < max_len:
            if word in vocabulary:
                idx = word_index[word]
                response_embedded[i][j] = embeddings_index[idx]

response_embedded.shape

(400, 43, 150)

In [10]:
one_hot_labels = to_categorical(labels, num_classes=3)

In [11]:
train_x, test_x, train_y, test_y = train_test_split(response_embedded, one_hot_labels, test_size=0.25, random_state=42)
print('training size:', train_x.shape[0])
print('testing size:', test_x.shape[0])

training size: 300
testing size: 100


In [12]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(max_len, vec_size))) # returns a sequence of vectors of dimension 32
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(128, return_sequences=True))) # returns a sequence of vectors of dimension 32
model.add(Dropout(0.5))
model.add(LSTM(256)) # return a single vector of dimension 32
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [13]:
learning_rate = [0.01]
epochs = 100
batch_size = 12

In [14]:
optimizer = optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_data=(test_x, test_y))

Train on 300 samples, validate on 100 samples
Epoch 1/100
300/300 [==============================] - 8s 26ms/step - loss: 1.4906 - acc: 0.4433 - val_loss: 2.2148 - val_acc: 0.3800
Epoch 2/100
300/300 [==============================] - 4s 12ms/step - loss: 1.1528 - acc: 0.5067 - val_loss: 1.3626 - val_acc: 0.5200
Epoch 3/100
300/300 [==============================] - 4s 13ms/step - loss: 1.0242 - acc: 0.5767 - val_loss: 0.9146 - val_acc: 0.5600
Epoch 4/100
300/300 [==============================] - 4s 13ms/step - loss: 0.9230 - acc: 0.5800 - val_loss: 1.3835 - val_acc: 0.3900
Epoch 5/100
300/300 [==============================] - 4s 14ms/step - loss: 0.9129 - acc: 0.6267 - val_loss: 1.1133 - val_acc: 0.4100
Epoch 6/100
300/300 [==============================] - 4s 14ms/step - loss: 0.9364 - acc: 0.5900 - val_loss: 1.1531 - val_acc: 0.5400
Epoch 7/100
300/300 [==============================] - 4s 15ms/step - loss: 0.9457 - acc: 0.5767 - val_loss: 0.9420 - val_acc: 0.4700
Epoch 8/100
300/

In [15]:
print('training error:', 1 - max(history.history['acc']), 'validation error:', 1 ß- max(history.history['val_acc']))

training error: 0.28 validation error: 0.3299999868869782


In [16]:
train_pred = model.predict(train_x)
train_k = cohen_kappa_score(pd.DataFrame(train_y).idxmax(axis=1), 
                            pd.DataFrame(train_pred).idxmax(axis=1), 
                            weights='quadratic')
test_pred = model.predict(test_x)
test_k = cohen_kappa_score(pd.DataFrame(test_y).idxmax(axis=1), 
                            pd.DataFrame(test_pred).idxmax(axis=1), 
                            weights='quadratic')

print('training kappa:', train_k, 'validation kappa:', test_k)


training kappa: 0.09963627690656485 validation kappa: 0.04423380726698267
